In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle
sys.path.append(os.getenv('PYTHONPATH')) 
import hcp_utils as hcp
from hcp_utils import parcellation_labels, view_parcellation
import matplotlib.pyplot as plt
from nilearn import plotting

#local
from src.utils.transforms import SelectROIs
from src.utils.helpers import get_fsLR32k_adjacency_matrix


In [ ]:
dataset_root = os.path.join(os.getenv('DATASETS_ROOT'), "MOSAIC")

In [ ]:
radius = 4
#load the whole brain adjacency dict for a desired radius. Note that this adjacency dict is not influenced by ROI selection or nan indices.
adjacency_filename = os.path.join(dataset_root, "decoding_artifacts",f"fsLR32k_adjacency_matrix_radius-{radius}.pkl")
if os.path.isfile(adjacency_filename):
    print(f"Adjacency dictionary of radius {radius} found. Loading...")
    with open(adjacency_filename, 'rb') as f:
        wb_adjacency_dict = pickle.load(f)
else:
    print(f"Creating adjacency dictionary with vertices of radius {radius}")
    wb_adjacency_dict = get_fsLR32k_adjacency_matrix(radius, save_flag=True)

In [ ]:
roi_indices = SelectROIs(selected_rois=["LO1"]).selected_roi_indices
vertex = roi_indices[0]
adjacent_vertices = wb_adjacency_dict[vertex]
print(adjacent_vertices)
stat=np.zeros((91282,))
stat[adjacent_vertices] = 2
stat[vertex] = 100 #center vertex

In [ ]:
"""
roi_indices = SelectROIs(selected_rois=["OFC"]).selected_roi_indices
not_real_adj = hcp.cortical_adjacency[roi_indices,:] 
adjacent_indices = np.argwhere(not_real_adj)[:,1]
stat=np.ones((91282,))
stat[adjacent_indices] = 100
stat[roi_indices] = 50
"""

In [ ]:
for hemi in ['left','right']:
    #flattened brain
    if hemi == 'left':
        cortex_data = hcp.left_cortex_data(stat)
        display = plotting.plot_surf_roi(hcp.mesh.flat_left, cortex_data,
        threshold=1, bg_map=hcp.mesh.sulc_left, colorbar=True, cmap='hot')
        plt.show()

    if hemi == 'right':
        cortex_data = hcp.right_cortex_data(stat)
        display = plotting.plot_surf_roi(hcp.mesh.flat_right, cortex_data,
        threshold=1,  bg_map=hcp.mesh.sulc_right, colorbar=True, cmap='hot')
        plt.show()

In [ ]:
view_parcellation(hcp.mesh.inflated, hcp.mmp)

In [ ]:
parcellation_labels(hcp.mmp)